# Spoken Dialogue System Lab: Integrating Rasa with Google Assitant

In this lab, we want to connect our dialogue system which is developed with Rasa to Google Home. In this way, as with saw previously with Giuliano on Alexa, we are going to exploit Google Assistant for its Automatic Speech Recognition (ASR) & Text-to-Speech (TTS) modules while all the other modules such as NLU,DM & NLG are handled by Rasa.
This lab is based on this [blog post](https://blog.rasa.com/going-beyond-hey-google-building-a-rasa-powered-google-assistant/).

## Step 1: Preparing Our Rasa Assistant

You can either practice this lab with the Rasa assistant available in the origin blog post, or edit the one you have practiced with Giuliano. In this notebook, we will get the assistant you connected to Alexa, and connect it to Google Home this time.

So <b>once again</b>, we need to clone the [repository](https://github.com/giTorto/rasa-restaurantbot) using git, or download the [zip](https://github.com/giTorto/rasa-restaurantbot/archive/master.zip) version.
As before, to train your RASA assistant, you can just run the command `rasa train` to train your dialogue system. To test your Dialogue System in the command line, use `rasa shell`.

So far it's easy right?

## Step 2: Initializing the Google Assistant Action

Now, we need to create a custom skill to use Google’s speech-to-text service. In this way, our custom skill is actually our Rasa assistant. Here is the to-do list:

1) Login to your google account and go to [Google Actions Console](https://console.actions.google.com/) and create a project by `Add Project` or `New Project`

Note: Remember which google account you are using at this step.

2) Name your project and pick the language you want your assistant to use. Then Click ‘Create project’. For this Notebook, I called my project `Restaurant Searcher`

3) Now, you will be asked to choose the category of your project. Scroll down to the bottom of the page and choose ‘Actions SDK’ option. By doing that, you tell your Google Assistant that you will implement your own custom action with a custom NLP and dialogue management system.

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://blog.rasa.com/content/images/2018/09/2018-09-10_22h15_26.png")

4) Once you choose `Actions SDK` and click OK, you should be in the overview page of the project. In the quick setup section, there is the option `Decide how your Action is invoked`. This is where you define the `Display Name` of the assistant for google. So that when You say this name, it activates your Rasa Assistant. 

In [3]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://blog.rasa.com/content/images/2018/09/2018-09-10_22h24_02.png")

For this notebook, I put the invocation name of `Restaurant Search Bot`.
So once again:

Project Name: `Restaurant Searcher`

Display Name: `Restaurant Search Bot`

 ## Step 3: Creating a Google Assistant action with Rasa as Engine

Now we need to create a Google action that invokes our Rasa assistant on user demand and passes all user inputs to Rasa after the skill is invoked. So we need two intents, i.e. `Invocation` intent and `Input Passing` intent.

For that, you need to:
1) Download gactions (Google Actions) from [Google Docs](https://developers.google.com/assistant/tools/gactions-cli)

2) Copy it in your project directory (and convert it to to executable file by `chmod +x gactions`).

3) Initialize Google Action Configuration by running `gactions init`

This should create a file named `action.json` (Not to be mistaken by `actions.py` which is for your Rasa Assistant)

Go to `action.json` and change its content to this below. Read the comments I have written below carefully.

In [ ]:
{
   "actions": [                                          #List of G-Actions that we need to have
      {
        "description": "Default Welcome Intent",        #The first action is welcome/Invocation
        "name": "MAIN",
        "fulfillment": {
          "conversationName": "welcome"
        },
        "intent": {
          "name": "actions.intent.MAIN",
          "trigger": {
            "queryPatterns":["talk to Restaurant Search Bot"]  #This is the string that invokes the Assitant
          }                                                    # Notice the display name is written here too
        }
      },
	  {
        "description": "Rasa Intent",                    # Second Action, the Input passing
        "name": "TEXT",
        "fulfillment": {
          "conversationName": "rasa_intent"
        },
        "intent": {
          "name": "actions.intent.TEXT",
          "trigger": {
            "queryPatterns":[]
          }
        }
      }],
    "conversations": {
      "welcome": {
        "name": "welcome",
        "url": "...",                        #This field is the address of our rasa, I.e. where to sent the inputs
        "fulfillmentApiVersion": 2           #WE WILL COME BACK TO IT AT STEP 5
    },
      "rasa_intent": {
        "name": "rasa_intent",
        "url": "...",                        #This field is the address of our rasa, I.e. where to sent the inputs
        "fulfillmentApiVersion": 2           #WE WILL COME BACK TO IT AT STEP 5
    }
  }
}

## Step 4: Creating a Rasa Connector to Google Assitant.

So by now you have a Rasa Assistant, and a Google Action that can be connected to a Rasa Assistant to use the ASR and TTS. Now, we should connect the two right?

For that, create a file called `ga_connector.py` and copy the code below in it. Read the comments I have written in the code carefully.  

In [ ]:
import logging    # These are neseccary libraries we are importing
import json
from sanic import Blueprint, response
from sanic.request import Request
from typing import Text, Optional, List, Dict, Any

from rasa.core.channels.channel import UserMessage, OutputChannel
from rasa.core.channels.channel import InputChannel
from rasa.core.channels.channel import CollectingOutputChannel

logger = logging.getLogger(__name__)

class GoogleConnector(InputChannel):
    """A custom http input channel.
    This implementation is the basis for a custom implementation of a chat
    frontend. You can customize this to send messages to Rasa Core and
    retrieve responses from the agent."""

    @classmethod
    def name(cls):
        return "google_assistant"


    def blueprint(self, on_new_message):                        # Here, we define the webhook that Google Assistant 
                                                                # will use to pass the user inputs to Rasa Core,    
        google_webhook = Blueprint('google_webhook', __name__)  # collect the responses and send them to Google Assistant

        @google_webhook.route("/", methods=['GET'])          
        async def health(request):                           # We design a Health route to control the connection 
            return response.json({"status": "ok"})           # is established by returning 200 ok message

        @google_webhook.route("/webhook", methods=['POST'])
        async def receive(request):                          # Then we define the main route for our purpose
            payload = request.json	
            intent = payload['inputs'][0]['intent'] 			
            text = payload['inputs'][0]['rawInputs'][0]['query'] 
            
            if intent == 'actions.intent.MAIN':	 #This is the initial message we ask to recieve when the assitant is invoked
                message = "Hello! Welcome to the Rasa-powered Google Assistant, Restaurant Search." 			 
            else:
                out = CollectingOutputChannel()			
                await on_new_message(UserMessage(text, out))
                responses = [m["text"] for m in out.messages]
                message = responses[0]
            r = {
                  "expectUserResponse": 'true',
                  "expectedInputs": [
                    {
                      "possibleIntents": [
                        {
                          "intent": "actions.intent.TEXT"   #This is our second intent defined in action.json, remember?
                        }
                    ],
                    "inputPrompt": {
                      "richInitialPrompt": {
                        "items": [
                          {
                            "simpleResponse": {
                              "textToSpeech": message,
                              "displayText": message
                              }
                            }
                          ]
                        }
                      }
                    }
                  ]
                }
            return response.json(r)				
        return google_webhook

## Step 5: Putting all the pieces together

Now lets put all of these together and have a Rasa Powered Google Assistant.
Ok lets start!

Things to do: 

1) Go to `credentials.yml` (if you dont have it, create this file in our project directory) and change its content to `ga_connector.GoogleConnector:` (If you are using Giuliano's, it should be `alexa_connector.AlexaConnector:`  )

2) Copy [ngrok](https://ngrok.com/) to your project directory and initialize it on a port, say `4747`, by running `./ngrok http 4747`

3) Run Rasa on the same port you occupied with ngrok by `rasa run --enable-api -p port` . So since I chose `4747` above, it will be `rasa run --enable-api -p 4747` 

Now what we did is basically running our Rasa agent on LOCAL port `4747` and asking `ngrok` for a path for this port from outside the local machine. As Giuliano told us, "Launching this command will create a unique URL that will be visible on your commandline. Copy the HTTPS URL, it should look like https://123abc4d.ngrok.io"

4) Copy the the URL `ngrok` produces, and add `/webhooks/google_assistant/webhook`. So, `https://123abc4d.ngrok.io` becomes `https://123abc4d.ngrok.io/webhooks/google_assistant/webhook`. Then, put this as URL in the fields I marked on `action.json` file we created at step 3. So it will be like this:

In [ ]:
{
   "actions": [                                          #List of G-Actions that we need to have
      {
        "description": "Default Welcome Intent",        #The first action is welcome/Invocation
        "name": "MAIN",
        "fulfillment": {
          "conversationName": "welcome"
        },
        "intent": {
          "name": "actions.intent.MAIN",
          "trigger": {
            "queryPatterns":["talk to Restaurant Search Bot"]  #This is the string that invokes the Assitant
          }
        }
      },
	  {
        "description": "Rasa Intent",                    # Second Action, the Input passing
        "name": "TEXT",
        "fulfillment": {
          "conversationName": "rasa_intent"
        },
        "intent": {
          "name": "actions.intent.TEXT",
          "trigger": {
            "queryPatterns":[]
          }
        }
      }],
    "conversations": {
      "welcome": {
        "name": "welcome",
        "url": "https://123abc4d.ngrok.io/webhooks/google_assistant/webhook",                        
        "fulfillmentApiVersion": 2           
    },
      "rasa_intent": {
        "name": "rasa_intent",
        "url": "https://123abc4d.ngrok.io/webhooks/google_assistant/webhook",           
        "fulfillmentApiVersion": 2      
    }
  }
}

5) On a seperate terminal run your rasa action server 

6) On another terminal run `gactions update --action_package action.json --project PROJECT_ID` for the variable `PROJECT_ID` go to [Google Cloud Console](https://console.cloud.google.com/) and copy it to the command above. it should be something like `restaurant-search-bot-123456`

By running this command, it asks you to access your google account (make sure you use the same account as step 2)
Once you grant access and authenticate, it should create a file called `creds.data` and return this:

`Your app for the Assistant for project restaurant-search-bot-12345 was successfully updated with your actions. Visit the Actions on Google console to finish registering your app and submit it for review at https://console.actions.google.com/project/restaurant-search-bot-12345/overview`

7) Now, run `gactions test --action_package action.json --project PROJECT_ID`. Dont forget to substitude `PROJECT_ID` .


Now you are done, You can test your assistant using your your mobile phone or Google Home devices. Try saying `Hey google, Talk to restaurant search bot`.